In [ ]:
# Import library and models
# NB: you need to import binary_model into the python path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Standard library
import numpy as np
import matplotlib.pyplot as plt
#!pip install tqdm 
from tqdm import tqdm

# The model
from model.binaryModel import binaryModel
from model.utils import initRandomArchitecture, initGaussianWeights, initUniformWeights, dataPath

In [ ]:
# The experiment
sim = 'statisticalAnalysis'
distribution_weight = 'normal'
experiment = 'display_phase_transition'
experiment = experiment + '_' + distribution_weight

N = 10000
duration = 2000
nbTrial = 10
K = 16 # Connectivity degree of the graph 
meanWeight = -0.1 # Average weights for the normal distribution
# List of standard deviation sigma (of the normal distribution)
sigmas_zoom = list([0.0607, 0.0614, 0.0621, 0.0629, 0.0636, 0.0643, 0.0643, 0.0657, 0.0664, 0.0671, 0.0679,
                 0.0686, 0.0693])
sigmas = list(np.arange(0.01, 0.06, 0.005))+\
        list(np.arange(0.06, 0.1, 0.01))+\
        sigmas_zoom+\
        list(np.arange(0.1, 0.2, 0.01))+\
        list(np.arange(0.2, 1, 0.1))+\
        list(np.arange(1, 11, 1))

print(f'The model will be evaluated on {len(sigmas)} values of sigma(W):')
print('sigma(W) =', sigmas)

In [ ]:
# Create the model
optionData = {'spikeCount':True, 'indexActive':False} # Option for saving data
brain = binaryModel(N, sim=sim, experiment=experiment)
seedConnectivity = 747
initRandomArchitecture(brain, K, seed=seedConnectivity) # Fixed architecture

In [ ]:
# Function to check if reservoir is ran already

def check_if_ran(sim, experiment, N, K, meanWeight):
    #Check network has not been run already
    pathBrain = dataPath(sim, experiment, N, K, meanWeight)
    file_path = pathBrain + f"/metadata_N{N}_K{K}*T{nbTrial}_W{meanWeight}_std{sigma}.npy"
    if os.path.exists(file_path):
        print('Network already run, next...\n \n')
        return True
    else:
        return False

In [ ]:
# Run the model

for sigma in tqdm(sigmas):
    sigma = np.round(sigma, 4)
    optionData = {'spikeCount':True, 'indexActive':False}
    for t in range(nbTrial):
        
        # Check if the network is already run
        check = check_if_ran(sim, experiment, N, K, meanWeight)
        if check:
            continue
        
        # Select weight distribution
        if distribution_weight == 'normal':
            initGaussianWeights(brain, meanWeight, sigma, seed=t*100)
        elif distribution_weight == 'uniform':
            initUniformWeights(brain, meanWeight, sigma, seed=t*100)
        
        # Run simulation
        brain.displayInfoNetwork()
        brain.initRandomState(0.2, seed=t*100) # Initialize the state of the reservoir
        brain.run(duration, **optionData)
        
        # Reset the network
        brain.reset()
    
    # Save data and metadata
    brain.saveData(**optionData)
    brain.saveMetadata()
    brain.reset(deep=True)


In [ ]:
# Analyse the data

from model.utils import statisticsActivity
import glob 

def statisticActivity_over_sigma(meanWeight, N, K, nbTrial, I=0.2,
                                 sim='statisticalAnalysis', experiment='allDegree'):
       
    result_dir = dataPath(sim, experiment)
    analysis_folder = os.path.join(result_dir, sim)
    if not os.path.isdir(analysis_folder):
        os.makedirs(analysis_folder)
        
    # Figure
    avgLabel = r'$<<A>_t>_T}$'
    varLabel = r'$<\sigma^2_t(A)/N>_T$'
    xlabel = r'$|\sigma^{\star}|$'
    # title1 = 'average over trial of the time averaged activity'
    # title2 = 'average over trial of the time variance activity'
    fig = plt.figure()
    ax = fig.add_subplot(2, 1, 1)
    #ax.set_xlabel(xlabel)
    ax.set_ylabel(avgLabel)
    ax.set_title('Average')
    ax1 = fig.add_subplot(2, 1, 2)
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(varLabel)
    ax1.set_title('Variance')
    
    avgAll = []
    varAll = []

    # data folder
    directory = dataPath(sim, experiment, N, K, meanWeights=meanWeight)
    filenames = glob.glob(directory + '/metadata_*T{0}*.npy'.format(nbTrial))
    stdWeights = []
    for filename in filenames:

        # Load datas
        metadata = np.load(filename, allow_pickle='TRUE').item()
        directoryActivity = metadata['dataPath']
        stdWeights.append(metadata['stdWeights'])
        fileData = metadata['spikeCountFile']
        duration = metadata['duration']
        idxStart = int(duration / 2)
        # Load activity data
        filepath = directoryActivity + '/' + fileData + '.npy'
        print(filepath)
        A_trials = np.load(filepath)

        # Global statistics
        avgTrials, varTrials = statisticsActivity(N, A_trials, idxStart)
        avgAll.append(avgTrials)
        varAll.append(varTrials)

    # plot average of all trials
    stdWeights = np.array(stdWeights)
    order = np.argsort(stdWeights)
    stdWeights = stdWeights[order]
    avgAll = np.array(avgAll)
    avgAll = avgAll[order]
    varAll = np.array(varAll)
    varAll = varAll[order]

    if meanWeight>0:
        marker = '^-'
        label = '$\sigma^{\star}>0$'
    elif meanWeight<0:
        marker = 'v-'
        label = '$\sigma^{\star}<0$'
    else:
        marker = 'o-'
        label = '$\mu=0$'
    
    if meanWeight != 0:
        x_axis = np.abs(stdWeights/meanWeight)
    else:
        x_axis = np.abs(stdWeights)

    ax.semilogx(x_axis, avgAll, marker, label=label)
    color = ax.get_lines()[-1].get_color()
    ax1.semilogx(x_axis, varAll, marker, color=color)

    plt.tight_layout()
    fig.savefig(analysis_folder + f'statActivity_sorted_N{N}_K{K}_d{duration}_T{nbTrial}.svg')


In [ ]:
# Run analysis
statisticActivity_over_sigma(meanWeight, N, K, nbTrial,
                             sim=sim, experiment=experiment)